In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Parkinsons_Disease/GSE202666'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Time-resolved RNA signatures of CD4+ T cells in Parkinson's disease [microRNAarray]"
!Series_summary	"We employed time-resolved microRNA expression analyses upon the activation of peripheral CD4+ T cells to track and functionally relate changes on cellular signaling in patients at different stages of PD"
!Series_overall_design	"A cohort of elderly male (age 53-85 yrs.; non-smokers), representative for different stages of Parkinson´s disease were tested (P1-5). Corresponding healthy controls (C1-5) (n=4) were matched for age (age 53-63 yrs.; non-smokers) and gender. One control was carried out twice (C1 and C1_2), representing independent replicates from the same healthy donor but from two different days of blood collection. CD4+ T cells were isolated from peripheral blood samples and were in vitro activated by αCD2/αCD3/αCD28 beads. Cellular samples were collected at 0, 2, 4, 8, 12 and 24 h after activation for subsequent RNA expression analyses."

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if dataset contains gene expression data
is_gene_available = True  # Based on series_description, it mentions RNA expression analyses

# Determine rows for 'Parkinsons_Disease', 'age', and 'gender'
trait_row = 0  # disease state: Parkinson's disease, Healthy Control
age_row = 3    # age: 53, 57, 63, etc.
gender_row = 2 # gender: male

# Define data type conversion functions
def convert_trait(value):
    value = value.split(': ')[1].strip()
    if value == "Parkinson's disease":
        return 1
    elif value == 'Healthy Control':
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(': ')[1].strip()
    if value == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE202666', './preprocessed/Parkinsons_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Parkinsons_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Parkinsons_Disease/trait_data/GSE202666.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6128103': [1.0, 53.0, 1.0], 'GSM6128104': [1.0, 53.0, 1.0], 'GSM6128105': [1.0, 53.0, 1.0], 'GSM6128106': [1.0, 53.0, 1.0], 'GSM6128107': [1.0, 53.0, 1.0], 'GSM6128108': [1.0, 53.0, 1.0], 'GSM6128109': [1.0, 57.0, 1.0], 'GSM6128110': [1.0, 57.0, 1.0], 'GSM6128111': [1.0, 57.0, 1.0], 'GSM6128112': [1.0, 57.0, 1.0], 'GSM6128113': [1.0, 57.0, 1.0], 'GSM6128114': [1.0, 57.0, 1.0], 'GSM6128115': [0.0, 63.0, 1.0], 'GSM6128116': [0.0, 63.0, 1.0], 'GSM6128117': [0.0, 63.0, 1.0], 'GSM6128118': [0.0, 63.0, 1.0], 'GSM6128119': [0.0, 63.0, 1.0], 'GSM6128120': [0.0, 63.0, 1.0], 'GSM6128121': [1.0, 75.0, 1.0], 'GSM6128122': [1.0, 75.0, 1.0], 'GSM6128123': [1.0, 75.0, 1.0], 'GSM6128124': [1.0, 75.0, 1.0], 'GSM6128125': [1.0, 75.0, 1.0], 'GSM6128126': [1.0, 75.0, 1.0], 'GSM6128127': [1.0, 85.0, 1.0], 'GSM6128128': [1.0, 85.0, 1.0], 'GSM6128129': [1.0, 85.0, 1.0], 'GSM6128130': [1.0, 85.0, 1.0], 'GSM6128131': [1.0, 85.0, 1.0], 'GSM6128132': [1.0, 85.0, 1.0], 'GSM6128133': [1.0, 76.0, 1.0], 'GSM612

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14',
       '15', '16', '17', '18', '21', '23', '24'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'SPOT_ID': ['miRNABrightCorner30', 'Blank', 'Blank', 'Blank', 'Blank'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'miRNA_ID': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Substep 1
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# Substep 2
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Substep 3
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Parkinsons_Disease/gene_data/GSE202666.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Parkinsons_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE202666', './preprocessed/Parkinsons_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Parkinsons_Disease/GSE202666.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
A new JSON file was created at: ./preprocessed/Parkinsons_Disease/cohort_info.json
